In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmazonVine-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     179886| RY01SAV7HZ8QO|B00NTI0CQ2|     667358431|Aketek 1080P LED ...|Home Entertainment|          4|            0|          0|   N|                Y|good enough for m...|not the best pict...| 2015-08-31|
|         US|   37293769|R1XX8SDGJ4MZ4L|B00BUCLVZU|     621695622|TiVo Mini with IR...|Home Entertainment|      

In [5]:
# Filter rows where total_votes is equal to or greater than 20
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
votes20 = df.filter("total_votes >= 20")

In [6]:
# Filter rows where number of helpful_votes divided by total_votes is equal to or greater than 50%
helpful_df = votes20.filter(votes20["helpful_votes"]/votes20["total_votes"] >= 0.5)
helpful_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24284435|R1VCSDR38XTALJ|B00WA42GTC|     464704031|Samsung Blu-ray P...|Home Entertainment|          1|           33|         40|   N|                N|Wouldn't recommen...|This Samsung BD-J...| 2015-08-31|
|         US|    5445862|R1KMC6B2JRCZ3D|B00SMBF3NS|     742282603|VIZIO E24-C1 E Se...|Home Entertainment|      

In [7]:
# Filter rows where a review was written as part of the Vine paid program
vine_paid = helpful_df.filter(helpful_df["vine"] == "Y")
vine_paid.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51288086|R3LLICBKRHLRJC|B00USEZ7W4|     202667718|Seiki SE32HYT  LE...|Home Entertainment|          4|           17|         22|   Y|                N|Great picture, a ...|Very nice, sharp ...| 2015-08-27|
|         US|   50853411| RUD135S06DEVE|B00ZIGRX4W|     471919351|Samsung JS7000, S...|Home Entertainment|      

In [8]:
# Filter rows where a review was written that was not part of the Vine paid program
vine_not_paid = helpful_df.filter(helpful_df["vine"] == "N")
vine_not_paid.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24284435|R1VCSDR38XTALJ|B00WA42GTC|     464704031|Samsung Blu-ray P...|Home Entertainment|          1|           33|         40|   N|                N|Wouldn't recommen...|This Samsung BD-J...| 2015-08-31|
|         US|    5445862|R1KMC6B2JRCZ3D|B00SMBF3NS|     742282603|VIZIO E24-C1 E Se...|Home Entertainment|      

In [19]:
# Determine the total number of all reviews
review_total = df.count()
review_total

705889

In [20]:
# Determine the total number of all 5-star reviews
total_fivestar = df.filter(df["star_rating"] == 5).count()
total_fivestar

373988

In [13]:
# Determine total number of paid reviews
paid_total = vine_paid.count()
paid_total

261

In [14]:
# Determine total number of unpaid reviews
unpaid_total = vine_not_paid.count()
unpaid_total

24040

In [15]:
# Determine total number of 5-star paid reviews
fivestar_paid = vine_paid.filter(vine_paid["star_rating"] == 5).count()
fivestar_paid

106

In [16]:
# Determine total number of 5-star unpaid reviews
fivestar_unpaid = vine_not_paid.filter(vine_paid["star_rating"] == 5).count()
fivestar_unpaid

10899

In [17]:
# Determine percentage of 5-star paid reviews
percentage_paid = (fivestar_paid / paid_total) * 100
percentage_paid

40.61302681992337

In [18]:
# Determine percentage of 5-star unpaid reviews
percentage_unpaid = (fivestar_unpaid / unpaid_total) * 100
percentage_unpaid

45.3369384359401